Download Flooding data

In [ ]:
from crawlerF.globalFloodDatabase import globalFloodDatabase
globalFloodDatabase().downloadAll("download")
# Finished on 2025/06/06

Download Population data

In [ ]:
from crawlerF.globalPopulation import globalPopulation
globalPopulation().downloadAll("test")
# Finished from 2025/06/07 to 2025/06/11

Mosaic Raster

In [ ]:
from raster.floodingMerge import floodingMerge
"""
Useful band
Band 1: flooded
Band 2: flood_duration
Band 5: jrc_perm_water (1 - permanent water, 0 - non-water)
"""
floodingMerge(r"C:\\0_PolyU\\flooding").mergeAll("test", 2)

Mosaic Population

In [ ]:
from raster.populationMerge import populationMerge
populationMerge(r"C:\\0_PolyU\\flooding").mergeAll("test", multiThread=2)